# Functions Utils

In [14]:
import numpy as np
from tqdm import tqdm
from scipy.io import mmread
from scipy.sparse import csr_matrix, tril, coo_matrix
from numpy import linalg
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from progetto_bis import main

# Execution

In [3]:
exec_data = main(['spa1.mtx'], [10**-4, 10**6])

Starting iterative jacobi method


C:\Users\s.crispino\AppData\Local\anaconda3\lib\site-packages\numpy\linalg\linalg.py:2154: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
  1%|▍                                                                             | 116/20000 [00:09<26:30, 12.50it/s]


**************************************************
Summary for jacobi method:
matrix_dimension = 1000
tol = 0.0001
max_iter = 20000
err_rel = 0.00165227651625821
iterations_number = 116
elapsed_time = 9.655620336532593
**************************************************
Starting iterative Gauß-Seidel method


  0%|                                                                               | 10/20000 [00:00<28:15, 11.79it/s]


**************************************************
Summary for Gauß-Seidel method:
matrix_dimension = 1000
tol = 0.0001
max_iter = 20000
err_rel = 0.009829657799574757
iterations_number = 10
elapsed_time = 1.121206521987915
**************************************************
Starting iterative gradient method


  1%|▌                                                                           | 144/20000 [00:00<00:08, 2307.17it/s]


**************************************************
Summary for gradient method:
matrix_dimension = 1000
tol = 0.0001
max_iter = 20000
err_rel = 0.03451187583863213
iterations_number = 144
elapsed_time = 0.06241416931152344
**************************************************
Starting iterative conjugate_gradient method


  1%|▌                                                                           | 144/20000 [00:00<00:10, 1850.85it/s]

**************************************************
Summary for conjugate_gradient method:
matrix_dimension = 1000
tol = 0.0001
max_iter = 20000
err_rel = 0.03451187583863213
iterations_number = 144
elapsed_time = 0.07780218124389648
**************************************************


# Plotting

## Processing Data for Plots

In [59]:
# ['jacobi', 'Gauß-Seidel', 'gradient', 'conjugate_gradient']
## ['iterations', 'matrix_dimension', 'tol', 'max_iter', 'err_rel', 'iterations_number', 'elapsed_time']
def processing_parameters_data(exec_data):
    data_plot = {}
    data_plot['param'] = []
    for method in exec_data.keys():
        data_plot[method] = []
        for param in ['err_rel', 'iterations_number', 'elapsed_time']:
            if len(data_plot['param']) < 3:
                data_plot['param'].append(param)
            data_plot[method].append(exec_data[method][param])


    df_plot = pd.DataFrame(data_plot)
    df_plot.head()
    
    return df_plot

## Plotting

In [60]:
def plotting_params(df_plot):
    for param in df_plot['param'].unique():

        df_plot[df_plot['param']== param].plot(kind='bar')

        plt.title(param)

        plt.plot()

## Plotting Convergence

In [61]:
# ['jacobi', 'Gauß-Seidel', 'gradient', 'conjugate_gradient']
def plotting_convergence(exec_data):
    for method in exec_data.keys():
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(exec_data[method]['iterations'].keys(), exec_data[method]['iterations'].values())
        
        
        plt.subplot(1, 2, 2)
        abs_values = [abs(val) for val in exec_data[method]['iterations'].values()]
        plt.plot(exec_data[method]['iterations'].keys(), abs_values)

### Tempo medio a iterazione?